# Create datasets
### Test set is defined af part of the original training datset; Due to hidden labels in the original test set provided by HuggingFace

In [1]:
from datasets import load_dataset, Dataset
from tqdm import tqdm
from more_itertools import roundrobin
import pyarrow as pa
import pandas as pd
import random 

/zhome/94/5/127021/miniconda3/envs/roberta_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Import dataset from HuggingFace
dataset = load_dataset("sst2")

Found cached dataset sst2 (/zhome/94/5/127021/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
100%|██████████| 3/3 [00:00<00:00, 195.86it/s]


In [3]:
# validation set
validation_dataset = dataset['validation']

In [4]:
# Check distribution in validation set
validation_dataset['label'].count(0) / validation_dataset['label'].count(1)

# Original test set size: 1821
# Thus, amount of 0-labels should be 895, while 1-labels should be 926

0.963963963963964

**Set random to shuffle training set**

In [23]:
N = len(dataset['train']['idx'])
random_list = list(range(N))
random.Random(42).shuffle(random_list)
print('N:',N,'\nrandom list:', len(random_list))

N: 67349 
random list: 67349


### Filter for test data

In [26]:
# Sample test set from original train data
# Using the same distribution as seen in the validation set

dataset_test_idx_0 = []
dataset_test_sentence_0 = []
dataset_test_label_0 = []

dataset_test_idx_1 = []
dataset_test_sentence_1 = []
dataset_test_label_1 = []

for i in random_list[:2000]:
    if len(dataset_test_label_0) == 895 and len(dataset_test_label_1) == 926:
        break
    if (dataset["train"]["label"][i] == 0):
        if len(dataset_test_label_0) < 895:
            dataset_test_idx_0.append(dataset["train"]["idx"][i])
            dataset_test_sentence_0.append(dataset["train"]["sentence"][i])
            dataset_test_label_0.append(dataset["train"]["label"][i])
    if (dataset["train"]["label"][i] == 1):
        if len(dataset_test_label_1) < 926:
            dataset_test_idx_1.append(dataset["train"]["idx"][i])
            dataset_test_sentence_1.append(dataset["train"]["sentence"][i])
            dataset_test_label_1.append(dataset["train"]["label"][i])
    

In [27]:
# Gather lists containing labels 0 and 1
dataset_test_idx = list(roundrobin(dataset_test_idx_0,dataset_test_idx_1))
dataset_test_sentence = list(roundrobin(dataset_test_sentence_0,dataset_test_sentence_1))
dataset_test_label = list(roundrobin(dataset_test_label_0,dataset_test_label_1))

#### Sample train data to not overlap test data

In [42]:

train_idx = list(set(dataset["train"]["idx"]) - set(dataset_test_idx))

In [65]:

dataset_train_label = [dataset["train"]["label"][i] for i in train_idx]


  0%|          | 189/65542 [03:51<22:12:44,  1.22s/it]


KeyboardInterrupt: 

In [63]:
dataset_train_sentence = [dataset["train"]["sentence"][i] for i in train_idx] 

[0, 0, 1, 0, 0]

In [33]:
dataset_train_idx = []
dataset_train_sentence = []
dataset_train_label = []

for i in tqdm(list(range(0,N))):
    if dataset['train']['idx'][i] not in dataset_test_idx:
        dataset_train_idx.append(dataset["train"]["idx"][i])
        dataset_train_sentence.append(dataset["train"]["sentence"][i])
        dataset_train_label.append(dataset["train"]["label"][i])

  6%|▋         | 4307/67349 [11:44<2:51:55,  6.11it/s]


KeyboardInterrupt: 

In [ ]:
# er der forskel på 
dataset_train_idx

In [ ]:
# og 
dataset_test_idx

### To datafram>dataset to save 

In [9]:
# lists as dictionary
test_df = pd.DataFrame({'idx': dataset_test_idx, 'sentence': dataset_test_sentence, 'label': dataset_test_label})
train_df = pd.DataFrame({'idx': dataset_train_idx, 'sentence': dataset_train_sentence, 'label': dataset_train_label})

# convert to Huggingface dataset
test_dataset = Dataset(pa.Table.from_pandas(test_df))
train_dataset = Dataset(pa.Table.from_pandas(train_df))

In [14]:
# Check distributions
print(test_dataset['label'].count(0) / test_dataset['label'].count(1))
print(validation_dataset['label'].count(0) / validation_dataset['label'].count(1))
print(train_dataset['label'].count(0) / train_dataset['label'].count(1))

0.9665226781857451
0.963963963963964
0.7926572374360518


In [15]:
# save datasets
train_dataset.save_to_disk('/work3/s174498/sst2_dataset/train_dataset')
validation_dataset.save_to_disk('/work3/s174498/sst2_dataset/validation_dataset')
test_dataset.save_to_disk('/work3/s174498/sst2_dataset/test_dataset')

Loading cached processed dataset at /zhome/a6/6/127219/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-137fd86d64e982b4.arrow


Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]